In [ ]:
import re
import pandas as pd
from nltk import tokenize
import pandas as pd
from nltk.tokenize import word_tokenize as wt
import nltk
import string
#nltk.download('averaged_perceptron_tagger')
#nltk.download('wordnet')
from nltk.corpus import wordnet
import ast
from nltk.tokenize import word_tokenize as wt
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
#Reading data
hgrd=pd.read_excel(r'C:\Users\rramn\Documents\Healthcare_analytics\physician_reviews_mining\healthgrades_latest.xlsx')
ratemd=pd.read_excel(r'C:\Users\rramn\Documents\Healthcare_analytics\physician_reviews_mining\Reviews\ratemds_reviews.xlsx')

#For Topic modelling
ratemdsent=pd.read_excel(r'C:\Users\rramn\Documents\Healthcare_analytics\physician_reviews_mining\Ratemds_vader_output.xlsx',sheet_name="both")
hgradesent=pd.read_excel(r'C:\Users\rramn\Documents\Healthcare_analytics\physician_reviews_mining\Healthgrades_vader_output.xlsx',sheet_name="both")

ratemd_sent=list(ratemdsent['Sentence'])
hgrade_sent=list(hgradesent['Sentence'])
# ratemd_docnames=[]


In [ ]:
nr,nc=hgrd.shape
nr1, nc1 = ratemd.shape
#Initialize POS, Lemmatizer, stop words 
def get_wordnet_pos(word):
    """Map POS tag to first character lemmatize() accepts"""
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}

    return tag_dict.get(tag, wordnet.NOUN)
with open(r'C:\Users\rramn\Documents\Healthcare_analytics\physician_reviews_mining\stopwords.txt', "r") as f:
    stop_words = f.read().splitlines()
print(stop_words)
# 1. Init Lemmatizer
lemmatizer = nltk.WordNetLemmatizer()

In [ ]:
#Preprocessing - Stop words removal, Lemmatization 
def preprocess(review):
        #processedReviews = []
    #for review in reviews:
        review = review.lower()
        review = " ".join(re.findall("[a-z]+", review))
        tokens = [lemmatizer.lemmatize(w, get_wordnet_pos(w)) for w in nltk.word_tokenize(review)]
        tokens = " ".join([item for item in tokens if item not in stop_words])
        #processedReviews.append(tokens)

        return tokens
sentrm=[]
senthg=[]
processedReviewsforRatemd = []
processedReviewsforHgrade = []
for i in range(len(ratemd_sent)):
    processedReviewsforRatemd.append(preprocess(ratemd_sent[i]))

for i in range(len(hgrade_sent)):
    processedReviewsforHgrade.append(preprocess(hgrade_sent[i]))
# processedReviewsforAll.append(preprocess(sentrm[0]))


In [ ]:
ratemdsent['Cleaned']=processedReviewsforRatemd
hgradesent['Cleaned']=processedReviewsforHgrade


# Corex implementation - Rate mds

#rate_rows=[]
#for f in range(len(ratemd_sent)):
#    rate_rows.append({"text":ratemd_sent[f]})
#df=pd.DataFrame(rate_rows)
df=ratemdsent
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(
    max_df=.5,
    min_df=10,
    max_features=None,
    ngram_range=(1, 2),
    norm=None,
    binary=True,
    use_idf=False,
    sublinear_tf=False
)
vectorizer = vectorizer.fit(df['Cleaned'])
tfidf = vectorizer.transform(df['Cleaned'])
vocab = vectorizer.get_feature_names()
print(len(vocab))

from corextopic import corextopic as ct

# Initializing
anchors = []
model = ct.Corex(n_hidden=6, seed=42)
model = model.fit(
    tfidf,
    words=vocab
)

In [ ]:
# Corex Topic modeling
for i, topic_ngrams in enumerate(model.get_topics(n_words=10)):
    topic_ngrams = [ngram[0] for ngram in topic_ngrams if ngram[1] > 0]
    print("Topic #{}: {}".format(i+1, ", ".join(topic_ngrams)))

In [ ]:
bg=pd.read_excel(r'C:\Users\rramn\Documents\Healthcare_analytics\physician_reviews_mining\Bag_ofwords.xlsx')
bg.info()
communication=list(bg.iloc[:,0])
expertise=list(bg.iloc[:,1])
time=list(bg.iloc[:,2])
bedside=list(bg.iloc[:,3])
office=list(bg.iloc[:,4])
cost=list(bg.iloc[:,5])

communication = [x for x in communication if str(x) != 'nan']
expertise = [x for x in expertise if str(x) != 'nan']
time = [x for x in time if str(x) != 'nan']
bedside = [x for x in bedside if str(x) != 'nan']
office = [x for x in office if str(x) != 'nan']
cost = [x for x in cost if str(x) != 'nan']

anchors = [
    communication,
    expertise,
    time,
    bedside,
    office,
    cost
]

model = ct.Corex(n_hidden=6, seed=42)
model = model.fit(
    tfidf,
    words=vocab,
    anchors=anchors, # Pass the anchors in here
    anchor_strength=3 # Tell the model how much it should rely on the anchors
)

for i, topic_ngrams in enumerate(model.get_topics(n_words=10)):
    topic_ngrams = [ngram[0] for ngram in topic_ngrams if ngram[1] > 0]
    print("Topic #{}: {}".format(i+1, ", ".join(topic_ngrams)))


In [ ]:
topic_df = pd.DataFrame(
    model.transform(tfidf), 
    columns=["topic_{}".format(i+1) for i in range(6)]
).astype(float)
topic_df.index = df.index
df = pd.concat([df, topic_df], axis=1)
summ=list(df['topic_1']+df['topic_2']+df['topic_3']+df['topic_4']+df['topic_5']+df['topic_6'])

In [ ]:
# Corex Topic modeling for Health grades
df1=hgradesent
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(
    max_df=.5,
    min_df=10,
    max_features=None,
    ngram_range=(1, 2),
    norm=None,
    binary=True,
    use_idf=False,
    sublinear_tf=False
)
vectorizer = vectorizer.fit(df1['Cleaned'])
tfidf = vectorizer.transform(df1['Cleaned'])
vocab = vectorizer.get_feature_names()
print(len(vocab))

from corextopic import corextopic as ct

# Initializing
anchors = []
model = ct.Corex(n_hidden=6, seed=42)
model = model.fit(
    tfidf,
    words=vocab
)

# Corex Topic modeling for Health grades
for i, topic_ngrams in enumerate(model.get_topics(n_words=10)):
    topic_ngrams = [ngram[0] for ngram in topic_ngrams if ngram[1] > 0]
    print("Topic #{}: {}".format(i+1, ", ".join(topic_ngrams)))


In [ ]:
# Corex Topic modeling for Health grades
anchors = [
    communication,
    expertise,
    time,
    bedside,
    office,
    cost
]
model = ct.Corex(n_hidden=6, seed=42)
model = model.fit(
    tfidf,
    words=vocab,
    anchors=anchors, # Pass the anchors in here
    anchor_strength=6 # Tell the model how much it should rely on the anchors
)

for i, topic_ngrams in enumerate(model.get_topics(n_words=10)):
    topic_ngrams = [ngram[0] for ngram in topic_ngrams if ngram[1] > 0]
    print("Topic #{}: {}".format(i+1, ", ".join(topic_ngrams)))


In [ ]:
topic_df = pd.DataFrame(
    model.transform(tfidf), 
    columns=["topic_{}".format(i+1) for i in range(6)]
).astype(float)
topic_df.index = df1.index
df1 = pd.concat([df1, topic_df], axis=1)

sumhg=list(df1['topic_1']+df1['topic_2']+df1['topic_3']+df1['topic_4']+df1['topic_5']+df1['topic_6'])

sum(x==0 for x in sumhg)


In [ ]:
#Writing both the results to excel file

df.to_excel('Ratemd_topic_result.xlsx',header=True)
df1.to_excel('Healthgrade_topic_result.xlsx',header=True)


In [ ]:
# Calculating the Control variables Words/ Review count, number of reviews, phrases/ review
wordcount=[]
reviewcount=[]
phrasecount=[]
for k in range(nr):
    stt=hgrd.iloc[k,7]
    wordcount.append(sum(word.strip(string.punctuation).isalpha() for word in stt.split()))
    reviewcount.append(len(ast.literal_eval(stt)))
    phrasecount.append(len(hgrd.iloc[k,8]))
hgrd['Word Count']=wordcount
hgrd['Review count']=reviewcount
hgrd['Phrase Count']=phrasecount

